In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import japanize_matplotlib
from IPython.core.display import display

import warnings
warnings.filterwarnings('ignore')

In [21]:
# train = pd.read_csv("csv/train.csv")
train = pd.read_csv("csv/processed_train.csv")
# test = pd.read_csv("csv/test.csv")
test = pd.read_csv("csv/processed_test.csv")
gender_submission = pd.read_csv("csv/gender_submission.csv")

In [22]:
sample_submission = pd.DataFrame(test["PassengerId"], columns=["PassengerId"])
train = pd.get_dummies(train, columns=["Sex", "Embarked"])
test = pd.get_dummies(test, columns=["Sex", "Embarked"])
train.drop(["PassengerId", "Name", "Cabin", "Ticket", "Title", "CategoricalAge", "CategoricalFare"], axis=1, inplace=True)
test.drop(["PassengerId", "Name", "Cabin", "Ticket", "Title"], axis=1, inplace=True)

display(train.head())

,Survived,Pclass,Age,SibSp,Parch,Fare,SibSp_0_1_2over,Parch_0_1_2_3over,FamilySize,IsAlone,Title_num,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,2,1,3,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,2,1,4,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,1,0,2,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,2,1,4,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,3,0,1,0,0,1


In [23]:
X_train = train.drop(["Survived"], axis=1)
y_train = train["Survived"]

In [24]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True)

score_list = []
models = []
for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print(f"fold{fold_+1} start")
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]

    lgb_train = lgb.Dataset(train_x,train_y)
    lgb_valid = lgb.Dataset(valid_x,valid_y)

    lgbm_params = {"objective":"binary"}

    gbm = lgb.train(
        params=lgbm_params,
        train_set=lgb_train,
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=20,
        verbose_eval=-1
    )
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_list.append(round(accuracy_score(valid_y, oof)*100, 2))
    models.append(gbm)
    print(f"fold{fold_+1} end\n")
print(score_list, "平均score", round(np.mean(score_list), 2))

fold1 start
[LightGBM] [Info] Number of positive: 232, number of negative: 362
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 220
[LightGBM] [Info] Number of data points in the train set: 594, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.390572 -> initscore=-0.444907
[LightGBM] [Info] Start training from score -0.444907
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [25]:
test_pred = np.zeros((len(test), 3))

for fold_,gbm in enumerate(models):
    pred_ = gbm.predict(test)
    test_pred[:,fold_] = pred_
pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
test_pred = (gbm.predict(test) > 0.5).astype(int)
sample_submission["Survived"] = test_pred
sample_submission.to_csv("3-fold_cross-validation_vol2.csv", index=False)